In [1]:
import os, sys
import torch
import numpy as np
from utils import word2sense, regression, listdir, image_to_tensor, Subject, cv_regression
from train import mean_condition_features
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
import scipy.stats as stats
from collections import OrderedDict 
import matplotlib.pyplot as plt
import copy
from feature_extractors import AlexNetConv5
import seaborn as sns


In [2]:
# Load visual features
alexnet = torch.load('features_conv5.pth') # dict with 1470 * 12544 features

alexnet_reshape = copy.deepcopy(alexnet)
for item in alexnet_reshape:
    alexnet_reshape[item] = np.reshape(alexnet_reshape[item], (256, 7, 7))
                             
vgg = torch.load('features_vgg16_conv5.pth')

In [3]:
# Load semantic embeddings
with open('ThingsWrd2Sns.txt', 'r', encoding='utf-8') as txt:
    temp = txt.readlines()
nDim = 2250
wordlist = [line.split(',')[0] for idx, line in enumerate(temp) if idx != 0] # create word list
word2sense = {new_list: np.zeros((1, nDim)) for new_list in wordlist} # create (word: embedding) dictionary
for i, line in enumerate(temp):
    if i == 0:
        continue
    embedding = line.split(',')
    embedding.remove('\n')
    embedding_float = ([float(j) for j in embedding[1:]])
   
    word2sense[embedding[0]] = np.array(embedding_float)
word2sense = OrderedDict(word2sense)

embeddings = word2sense



In [4]:
# Wordvec filtered by CM
wordvec = {}
with open('ThingsWrd2Vec_subset.txt', 'r', encoding='utf-8') as txt:
    temp = txt.readlines()

for i, wv in enumerate(temp):
    if i == 0:
        continue
    w = (temp[i].split(','))[0]
    if w in word2sense:
        wordvec[w] = temp[i].split(',')[1:]
        wordvec[w][-1] = wordvec[w][-1].replace('\n', '')

for wv in wordvec:
    wordvec[wv] = np.array(wordvec[wv]).astype(np.float)

embeddings = wordvec
features = {item: alexnet[item] for item in alexnet.keys() if item in wordvec}

In [ ]:
print([item for item in alexnet.keys() if item in wordvec])

In [10]:
featsize = vgg['/home/chan21/projects/semanticdimensionality/kh/object2vec_encoder_python/images/aardvark'].shape[0]
nfeat = 512

In [ ]:
# 1-a. Let's cross-validate visual -> semantic regression, with unit removal

acc = []
kf = KFold(n_splits = 9)


for i in range(256):
    alexnet_temp = copy.deepcopy(alexnet_reshape)
    for item in alexnet_temp:
        alexnet_temp[item][i][:][:] = 0
    rs = []
    pred = []
    test = []
    for train_index, test_index in kf.split(np.stack([np.reshape(feature, (featsize,)) for feature in alexnet_temp.values()])):
        train_regressor = np.stack([np.reshape(feature, (featsize,)) for i, feature in enumerate(alexnet_temp.values()) if i in train_index])
        test_regressor = np.stack([np.reshape(feature, (featsize,)) for i, feature in enumerate(alexnet_temp.values()) if i in test_index])
        train_predictor = np.stack([embedding for i, embedding in enumerate(embeddings.values()) if i in train_index])
        test_predictor = np.stack([embedding for i, embedding in enumerate(embeddings.values()) if i in test_index])
        regr = Ridge(alpha=0, fit_intercept=False)
        regr.fit(train_regressor, train_predictor)
        pred_predictor = regr.predict(test_regressor) 
        pred.append(pred_predictor)
        test.append(test_predictor) 
    preds = np.vstack([p for p in pred])
    tests = np.vstack([t for t in test])

    for dim in range(preds.shape[1]): 
        r, _ = stats.pearsonr(preds[:, dim], tests[:, dim]) # compute voxelwise Pearson correlation 
        rs.append(r)   

    plt.figure(figsize=(10,5))
    plt.bar(range(0, 2250), rs)
    mean_r = np.nanmean(np.array(rs), axis=0)
    print(mean_r)
    #acc.append(mean_r)

In [ ]:
# 1-b. Now, let's cv visual -> Object2vec, fMRI with unit removal.
subjects = [1, 2, 3, 4]
rois = ['LOC']
for roi in rois:
    sub_voxel_regressor = {}
    for subj in subjects:
        subject = Subject(subj, [roi])
        voxel_regressor = np.stack([condition_voxel for condition, condition_voxel in (subject.condition_voxels).items()])
        sub_voxel_regressor[subj] = voxel_regressor

        #print(sub_voxel_regressor[subj].shape)
    all_voxel_regressor = np.hstack([sub_voxel_regressor[s] for s in subjects])

acc = []
kf = KFold(n_splits = 9)

feat_extractor = AlexNetConv5()
c_feat = mean_condition_features(feat_extractor, nfeat)

c_feat_reshape = copy.deepcopy(c_feat)
for item in c_feat_reshape:
    c_feat_reshape[item] = np.reshape(c_feat_reshape[item], (nfeat, 11, 11))

for i in range(mfeat):
    c_feat_temp = copy.deepcopy(c_feat_reshape)
    for item in c_feat_temp:
        c_feat_temp[item][i][:][:] = 0
    rs = []
    pred = []
    test = []
    for train_index, test_index in kf.split(np.stack([np.reshape(feature, (12544,)) for feature in c_feat_temp.values()])):
        train_regressor = np.stack([np.reshape(feature, (12544,)) for i, feature in enumerate(c_feat_temp.values()) if i in train_index])
        test_regressor = np.stack([np.reshape(feature, (12544,)) for i, feature in enumerate(c_feat_temp.values()) if i in test_index])
        train_predictor = all_voxel_regressor[train_index, :]
        test_predictor = all_voxel_regressor[test_index, :]
        regr = Ridge(alpha=0, fit_intercept=False)
        regr.fit(train_regressor, train_predictor)
        pred_predictor = regr.predict(test_regressor) 
        pred.append(pred_predictor)
        test.append(test_predictor) 
    preds = np.vstack([p for p in pred])
    tests = np.vstack([t for t in test])

    for dim in range(preds.shape[1]): 
        r, _ = stats.pearsonr(preds[:, dim], tests[:, dim]) # compute voxelwise Pearson correlation 
        rs.append(r)   

    mean_r = np.nanmean(np.array(rs), axis=0)
    print(mean_r)
    acc.append(mean_r)



In [ ]:
print(acc[255], acc[9])

In [ ]:
# cv regression with Things fMRI
subjects = ['subj001/', 'subj002/', 'subj003/', 'subj004/']
rois = ['LOC']
layers = ['conv5']
predictpath = {
    "alexnetreal": "predictedww/",
    "alexnetrandom": "predicted1026weight2/",
    "vggreal": "predictedvgg/",
}
weight = "alexnetreal"
for roi in rois:
    for layer in layers:
        sub_voxel_regressor = {}
        for subj in subjects:
            path = predictpath[weight]
            fmri_path = path + subj + roi + "_" + layer
            fmri_path = os.path.join(os.getcwd(), fmri_path) 

            conditions = sorted(listdir(fmri_path))
            condition_voxels = {}
            for condition in conditions:
                if condition.split('/')[-1] in alexnet.keys():
                    file_name = listdir((os.path.join(fmri_path, condition)))[0]
                    file_path = os.path.join(fmri_path, condition, file_name)
                    condition_voxels[condition] = np.load(file_path) 
            voxel_regressor = np.stack([condition_voxel for condition, condition_voxel in OrderedDict(condition_voxels).items()])
            sub_voxel_regressor[subj] = voxel_regressor

            #print(sub_voxel_regressor[subj].shape)
        all_voxel_regressor = np.hstack([sub_voxel_regressor[s] for s in subjects])


In [ ]:
# load rank by vis->sem
acc = np.load('./UnitRemoval/r_vis2sem.npy')

In [ ]:
# save (load) alexnet features for object2vec
feat_extractor = AlexNetConv5()
c_feat = mean_condition_features(feat_extractor, 256)

c_feat_reshape = copy.deepcopy(c_feat)
for item in c_feat_reshape:
    c_feat_reshape[item] = np.reshape(c_feat_reshape[item], (256, 7, 7))


In [ ]:
# cv regression with Object2vec fMRI
subjects = [1, 2, 3, 4]
rois = ['LOC']
for roi in rois:
    sub_voxel_regressor = {}
    for subj in subjects:
        subject = Subject(subj, [roi])
        voxel_regressor = np.stack([condition_voxel for condition, condition_voxel in (subject.condition_voxels).items()])
        sub_voxel_regressor[subj] = voxel_regressor

        #print(sub_voxel_regressor[subj].shape)
    all_voxel_regressor = np.hstack([sub_voxel_regressor[s] for s in subjects])

rs = []
pred = []
test = []
kf = KFold(n_splits = 9)
for train_index, test_index in kf.split(np.stack([np.reshape(feature, (12544,)) for feature in c_feat.values()])):
    train_regressor = np.stack([np.reshape(feature, (12544,)) for i, feature in enumerate(c_feat.values()) if i in train_index])
    test_regressor = np.stack([np.reshape(feature, (12544,)) for i, feature in enumerate(c_feat.values()) if i in test_index])
    train_predictor = all_voxel_regressor[train_index, :]
    test_predictor = all_voxel_regressor[test_index, :]
    regr = Ridge(alpha=0, fit_intercept=False)
    regr.fit(train_regressor, train_predictor)
    pred_predictor = regr.predict(test_regressor) 
    pred.append(pred_predictor)
    test.append(test_predictor) 
preds = np.vstack([p for p in pred])
tests = np.vstack([t for t in test])
for dim in range(preds.shape[1]): 
    r, _ = stats.pearsonr(preds[:, dim], tests[:, dim]) # compute voxelwise Pearson correlation 
    rs.append(r)   

mean_r_baseline = np.nanmean(np.array(rs), axis=0)


In [ ]:
np.linspace(0, 256, 257)

In [ ]:
# Let's see the unit removal effects based on semantic richness on: 
# 1-a-i. fMRI encoding - Object2vec

acc_fmri = []
for i in range(0, 256, 1):
    alexnet_temp = copy.deepcopy(alexnet_reshape)
    maxidx = np.argsort(np.array(acc))[:i]
    maxidx = (np.linspace(0, 256, 257))[:i]
    if i == 0:
        acc_fmri.append(mean_r_baseline)
    else:
        for n in maxidx:
            for item in c_feat_temp:
                c_feat_temp[item][n][:][:] = 0
    rs = []
    pred = []
    test = []
    for train_index, test_index in kf.split(np.stack([np.reshape(feature, (12544,)) for feature in alexnet_temp.values()])):
        train_regressor = np.stack([np.reshape(feature, (12544,)) for i, feature in enumerate(alexnet_temp.values()) if i in train_index])
        test_regressor = np.stack([np.reshape(feature, (12544,)) for i, feature in enumerate(alexnet_temp.values()) if i in test_index])
        train_predictor = all_voxel_regressor[train_index, :]
        test_predictor = all_voxel_regressor[test_index, :]
        regr = Ridge(alpha=0, fit_intercept=False)
        regr.fit(train_regressor, train_predictor)
        pred_predictor = regr.predict(test_regressor) 
        pred.append(pred_predictor)
        test.append(test_predictor) 
    preds = np.vstack([p for p in pred])
    tests = np.vstack([t for t in test])

    for dim in range(preds.shape[1]): 
        r, _ = stats.pearsonr(preds[:, dim], tests[:, dim]) # compute voxelwise Pearson correlation 
        rs.append(r)   
    
    mean_r = np.nanmean(np.array(rs), axis=0)
    acc_fmri.append(mean_r)
    
# x = np.linspace(0, 257, 257)
# title_font = {'fontname':'DejaVu Sans', 'size':'15', 'color':'black', 'weight':'normal',
#       'verticalalignment':'bottom'} # Bottom vertical alignment for more space
# axis_font = {'fontname':'DejaVu Sans', 'size':'10'}
# sns.set()
# plt.figure(figsize=(10,5))
# plt.xlabel('Number of visual features removed (ranked by predictability on semantic embeddings)', **axis_font)
# plt.ylabel('Prediction accuracy on fMRI response (Object2vec)', **axis_font)
# plt.title("Linear regression: x (AlexNet Conv5 visual features), y (fMRI response)", **title_font)
# plt.plot(x, np.array(acc_fmri))
# sns.reset_defaults()

In [ ]:
# Let's see the unit removal effects based on semantic richness on: 
# 1-a-i. fMRI encoding - Object2vec (sequential)
feat_extractor = AlexNetConv5()
c_feat = mean_condition_features(feat_extractor, 256)

c_feat_reshape = copy.deepcopy(c_feat)
for item in c_feat_reshape:
    c_feat_reshape[item] = np.reshape(c_feat_reshape[item], (256, 7, 7))

acc_fmri_seq = []
for u in range(0, 256, 1):
    c_feat_temp = copy.deepcopy(c_feat_reshape)
    maxidx = np.arange(0, 256, 1)[:u]
    for n in maxidx:
        for item in c_feat_temp:
            c_feat_temp[item][n][:][:] = 0
    rs = []
    pred = []
    test = []
    for train_index, test_index in kf.split(np.stack([np.reshape(feature, (12544,)) for feature in c_feat_temp.values()])):
        train_regressor = np.stack([np.reshape(feature, (12544,)) for i, feature in enumerate(c_feat_temp.values()) if i in train_index])
        test_regressor = np.stack([np.reshape(feature, (12544,)) for i, feature in enumerate(c_feat_temp.values()) if i in test_index])
        train_predictor = all_voxel_regressor[train_index, :]
        test_predictor = all_voxel_regressor[test_index, :]
        regr = Ridge(alpha=0, fit_intercept=False)
        regr.fit(train_regressor, train_predictor)
        pred_predictor = regr.predict(test_regressor) 
        pred.append(pred_predictor)
        test.append(test_predictor) 
    preds = np.vstack([p for p in pred])
    tests = np.vstack([t for t in test])

    for dim in range(preds.shape[1]): 
        r, _ = stats.pearsonr(preds[:, dim], tests[:, dim]) # compute voxelwise Pearson correlation 
        rs.append(r)   
    
    mean_r = np.nanmean(np.array(rs), axis=0)
    acc_fmri_seq.append(mean_r)
    


In [ ]:
import random

# Let's see the unit removal effects based on semantic richness on: 
# 1-a-i. fMRI encoding - Object2vec (random)
feat_extractor = AlexNetConv5()
c_feat = mean_condition_features(feat_extractor, 256)

c_feat_reshape = copy.deepcopy(c_feat)
for item in c_feat_reshape:
    c_feat_reshape[item] = np.reshape(c_feat_reshape[item], (256, 7, 7))

acc_fmri_random = []

l = random.sample(list(np.arange(0, 256, 1)), 256)
for u in range(0, 256, 1):
    c_feat_temp = copy.deepcopy(c_feat_reshape)
    
    maxidx = l[:u]
    print(maxidx)
    for n in maxidx:
        for item in c_feat_temp:
            c_feat_temp[item][n][:][:] = 0
    rs = []
    pred = []
    test = []
    for train_index, test_index in kf.split(np.stack([np.reshape(feature, (12544,)) for feature in c_feat_temp.values()])):
        train_regressor = np.stack([np.reshape(feature, (12544,)) for i, feature in enumerate(c_feat_temp.values()) if i in train_index])
        test_regressor = np.stack([np.reshape(feature, (12544,)) for i, feature in enumerate(c_feat_temp.values()) if i in test_index])
        train_predictor = all_voxel_regressor[train_index, :]
        test_predictor = all_voxel_regressor[test_index, :]
        regr = Ridge(alpha=0, fit_intercept=False)
        regr.fit(train_regressor, train_predictor)
        pred_predictor = regr.predict(test_regressor) 
        pred.append(pred_predictor)
        test.append(test_predictor) 
    preds = np.vstack([p for p in pred])
    tests = np.vstack([t for t in test])

    for dim in range(preds.shape[1]): 
        r, _ = stats.pearsonr(preds[:, dim], tests[:, dim]) # compute voxelwise Pearson correlation 
        rs.append(r)   
    
    mean_r = np.nanmean(np.array(rs), axis=0)
    acc_fmri_random.append(mean_r)
    


In [ ]:
acc_fmri = np.load("UnitRemoval/r_vis2sem_1fmri.npy")
x = np.linspace(0, 257, 257)
title_font = {'fontname':'DejaVu Sans', 'size':'15', 'color':'black', 'weight':'normal',
      'verticalalignment':'bottom'} # Bottom vertical alignment for more space
axis_font = {'fontname':'DejaVu Sans', 'size':'10'}
sns.set()
plt.figure(figsize=(10,5))
plt.xlabel('Number of visual features removed (ranked by predictability on semantic embeddings)', **axis_font)
plt.ylabel('Prediction accuracy on fMRI response (Object2vec)', **axis_font)
plt.title("Linear regression: x (AlexNet Conv5 visual features), y (fMRI response)", **title_font)
plt.plot(x, np.array(acc_fmri), color='red', label='ablation by prediction accuracy')
x = np.linspace(0, 257, 256)

plt.plot(x, np.array(acc_fmri_seq), color='blue', label='sequential ablation')
plt.plot(x, np.array(acc_fmri_random), color='green', label='random ablation')

plt.legend(prop={'size': 14});

sns.reset_defaults()

In [ ]:
len(np.argsort(np.array(acc))[:255])

In [ ]:
# Let's see the unit removal effects based on fMRI prediction on: 
# 1-b-i. semantic embedding

acc_ = []
for u in range(0, 256, 1):
    alexnet_temp = copy.deepcopy(alexnet)
    maxidx = np.argsort(np.array(acc))[:u]
    for n in maxidx:
        for item in alexnet_temp:
            alexnet_temp[item][n][:][:] = 0
    rs = []
    pred = []
    test = []
    for train_index, test_index in kf.split(np.stack([np.reshape(feature, (12544,)) for feature in alexnet_temp.values()])):
        train_regressor = np.stack([np.reshape(feature, (12544,)) for i, feature in enumerate(alexnet_temp.values()) if i in train_index])
        test_regressor = np.stack([np.reshape(feature, (12544,)) for i, feature in enumerate(alexnet_temp.values()) if i in test_index])
        train_predictor = np.stack([embedding for i, embedding in enumerate(word2sense.values()) if i in train_index])
        test_predictor = np.stack([embedding for i, embedding in enumerate(word2sense.values()) if i in test_index])
        regr = Ridge(alpha=0, fit_intercept=False)
        regr.fit(train_regressor, train_predictor)
        pred_predictor = regr.predict(test_regressor) 
        pred.append(pred_predictor)
        test.append(test_predictor) 
    preds = np.vstack([p for p in pred])
    tests = np.vstack([t for t in test])

    for dim in range(preds.shape[1]):
        r, _ = stats.pearsonr(preds[:, dim], tests[:, dim]) # compute voxelwise Pearson correlation 
        rs.append(r)   
    
    mean_r = np.nanmean(np.array(rs), axis=0)
    acc_.append(mean_r)


In [ ]:
# Let's see the unit removal effects based on fMRI prediction on: 
# 1-b-i. semantic embedding (sequential, random)

acc_random = []

l = random.sample(list(np.arange(0, 256, 1)), 256)
    
for u in range(0, 256, 1):
    alexnet_temp = copy.deepcopy(alexnet)
    #maxidx = np.argsort(np.array(acc))[:u]
    #maxidx = np.arange(0, 256, 1)[:u]
    maxidx = l[:u]

    for n in maxidx:
        for item in alexnet_temp:
            alexnet_temp[item][n][:][:] = 0
    rs = []
    pred = []
    test = []
    for train_index, test_index in kf.split(np.stack([np.reshape(feature, (12544,)) for feature in alexnet_temp.values()])):
        train_regressor = np.stack([np.reshape(feature, (12544,)) for i, feature in enumerate(alexnet_temp.values()) if i in train_index])
        test_regressor = np.stack([np.reshape(feature, (12544,)) for i, feature in enumerate(alexnet_temp.values()) if i in test_index])
        train_predictor = np.stack([embedding for i, embedding in enumerate(word2sense.values()) if i in train_index])
        test_predictor = np.stack([embedding for i, embedding in enumerate(word2sense.values()) if i in test_index])
        regr = Ridge(alpha=0, fit_intercept=False)
        regr.fit(train_regressor, train_predictor)
        pred_predictor = regr.predict(test_regressor) 
        pred.append(pred_predictor)
        test.append(test_predictor) 
    preds = np.vstack([p for p in pred])
    tests = np.vstack([t for t in test])

    for dim in range(preds.shape[1]):
        r, _ = stats.pearsonr(preds[:, dim], tests[:, dim]) # compute voxelwise Pearson correlation 
        rs.append(r)   
    
    mean_r = np.nanmean(np.array(rs), axis=0)
    #acc_seq.append(mean_r)
    acc_random.append(mean_r)

In [ ]:
np.save("UnitRemoval/r_vis2fmri_1sem", acc_)
x = np.linspace(0, 256, 256)
title_font = {'fontname':'DejaVu Sans', 'size':'15', 'color':'black', 'weight':'normal',
      'verticalalignment':'bottom'} # Bottom vertical alignment for more space
axis_font = {'fontname':'DejaVu Sans', 'size':'10'}
sns.set()
plt.figure(figsize=(10,5))
plt.xlabel('Number of visual features removed (ranked by predictability on fMRI response)', **axis_font)
plt.ylabel('Prediction accuracy on semantic embeddings (Things)', **axis_font)
plt.title("Linear regression: x (AlexNet Conv5 visual features), y (Word2sense)", **title_font)
plt.plot(x, np.array(acc_), color='red', label='ablation by prediction accuracy')
plt.plot(x, np.array(acc_seq), color='blue', label='sequential ablation')
plt.plot(x, np.array(acc_random), color='green', label='random ablation')

plt.legend(prop={'size': 14});

sns.reset_defaults()

In [ ]:
# (2) classification - Things category prediction with linear classifier


for i in range(1, 256, 1):
    alexnet_temp = copy.deepcopy(alexnet_reshape)
    maxidx = np.argsort(np.array(acc))[:i]
    for n in maxidx:
        for item in alexnet_temp:
            alexnet_temp[item][n][:][:] = 0
            
